Importar bibliotecas

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import datasets, metrics
from mealpy.bio_based import SMA 
import numpy as np

ImportError: cannot import name 'SMA' from 'mealpy.swarm_based' (C:\Users\Giuli\AppData\Roaming\Python\Python312\site-packages\mealpy\swarm_based\__init__.py)

Ler o arquivo CSV

In [7]:

df = pd.read_csv('../ocidental_data.csv')
df

,time,dhi_pyr,ghi_pyr,air_temperature,relative_humidity,barometric_pressure,precipitation
0,2016-01-10 00:00:00,0.0,0.0,26.716667,93.900000,1011.0,0.0
1,2016-01-10 00:01:00,0.0,0.0,26.723333,93.833333,1011.0,0.0
2,2016-01-10 00:02:00,0.0,0.0,26.720000,93.600000,1011.0,0.0
3,2016-01-10 00:03:00,0.0,0.0,26.720000,93.566667,1011.0,0.0
4,2016-01-10 00:04:00,0.0,0.0,26.733333,93.700000,1011.0,0.0
...,...,...,...,...,...,...,...
1356475,2023-12-05 23:56:00,0.0,0.0,24.800000,92.700000,1011.0,0.0
1356476,2023-12-05 23:57:00,0.0,0.0,24.800000,92.700000,1011.0,0.0
1356477,2023-12-05 23:58:00,0.0,0.0,24.700000,92.600000,1011.0,0.0
1356478,2023-12-05 23:59:00,0.0,0.0,24.700000,92.600000,1011.0,0.0


Processar os dados, filtrando apenas as colunas de interesse

In [8]:
df = df[['time', 'air_temperature', 'relative_humidity', 'dhi_pyr']]

Remove valores inválidos (faltantes, N/A, NULL) e valores dentro de um intervalo de tempo noturno, no qual não há incidência solar

In [9]:
df.dropna(subset=['air_temperature', 'relative_humidity', 'dhi_pyr'], inplace=True)

df.time = pd.to_datetime(df['time'])
df.time

clean_df = df[df.time.dt.strftime('%H:%M').between('06:00','19:00')]
df = clean_df
df


C:\Users\Giuli\AppData\Local\Temp\ipykernel_26856\1648498306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['air_temperature', 'relative_humidity', 'dhi_pyr'], inplace=True)
C:\Users\Giuli\AppData\Local\Temp\ipykernel_26856\1648498306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time = pd.to_datetime(df['time'])


,time,air_temperature,relative_humidity,dhi_pyr
360,2016-01-10 06:00:00,25.033333,98.833333,0.0
361,2016-01-10 06:01:00,25.030000,98.800000,0.0
362,2016-01-10 06:02:00,25.020000,98.866667,0.0
363,2016-01-10 06:03:00,25.010000,98.900000,0.0
364,2016-01-10 06:04:00,25.006667,98.900000,0.0
...,...,...,...,...
1356175,2023-12-05 18:56:00,27.900000,84.600000,0.5
1356176,2023-12-05 18:57:00,27.900000,85.000000,0.0
1356177,2023-12-05 18:58:00,27.800000,85.500000,0.0
1356178,2023-12-05 18:59:00,27.700000,85.900000,0.0


Definir as variáveis e o valor a ser calculado

In [10]:
X = df[['air_temperature', 'relative_humidity']]
y = df['dhi_pyr']


Dividir os dados em treino e teste (80% treino, 20% teste)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalização dos dados

In [12]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [13]:
KERNEL_ENCODER = LabelEncoder()
KERNEL_ENCODER.fit(['linear', 'poly', 'rbf', 'sigmoid'])

LabelEncoder()

Inicializar e treinar o modelo Random Forest

In [14]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_std, y_train.values.ravel())

RandomForestRegressor(random_state=42)

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
def fitness_function(solution, flg = None):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid

#     kernel_encoded = round(solution[0])
#     c = solution[1]
#     kernel_decoded = KERNEL_ENCODER.inverse_transform([kernel_encoded])[0]

#     svc = SVC(C=c, random_state=1, kernel=kernel_decoded, max_iter=100)
    rfr = RandomForestRegressor(max_depth = round(solution[1]), n_estimators=round(solution[0]))
    # Fit the model
#     svc.fit(X_train_std, y_train.values.ravel())
    # Make the predictions
#     y_predict = svc.predict(X_test_std)
    if(flg == 'run'):
        rfr.fit(X_train_std, y_train.values.ravel())
        return rfr
    # Measure the performance
#     x_print = cross_val_score(rfr, X_train_std, y_train.values.ravel(), cv=3).mean()
    accuracy = cross_validate(rfr, X_train_std, y_train.values.ravel(), cv=5, scoring='max_error', error_score='raise')
    result = np.array(accuracy['test_score']).mean()
    return abs(result)

problem = {
    "fit_func": fitness_function,
    "lb": [  1,  11],
    "ub": [500, 50],
    "minmax": "min",
}

In [48]:
fitness_function([20, 10])

np.float64(460.14155690030145)

In [44]:
# from mealpy.swarm_based import BaseSMA
import mealpy
# from mealpy.bio_based.SMA import BaseSMA as SMA
from mealpy.bio_based.SMA import DevSMA


# import mealpy.bio_based.SMA as SMA

model = DevSMA(epoch=5, pop_size=20)
model.solve(problem)
print(f"Best solution: {model.solution[0]}")
# print(f"Best kernel: {KERNEL_ENCODER.inverse_transform([int(model.solution[0][0])])[0]}, Best c: {model.solution[0][1]}")

print(f"Best accuracy: {model.solution[1]}")

TypeError: Problem.__init__() missing 1 required positional argument: 'bounds'

In [45]:
melhor_modelo = fitness_function(model.solution[0], flg='run')
y_pred = melhor_modelo.predict(X_test_std)

AttributeError: 'DevSMA' object has no attribute 'solution'

In [ ]:
# import pylab as plt
import matplotlib.pyplot as plt

plt.figure(dpi=150)

plt.plot(y_test, y_test, 'k-')
plt.plot(y_test[::30], y_pred[::30], 'b.', linewidth=0.2)

plt.show()

In [ ]:
model.solution

In [ ]:
# Example to print or store metrics after running the model
import numpy as np

# Assuming `model.history` contains the best solution of each iteration (if available in Mealpy)
if hasattr(model, 'history'):
    best_fitness_per_iteration = model.history['fitness']
    mean_fitness = np.mean(best_fitness_per_iteration)
    std_fitness = np.std(best_fitness_per_iteration)
    
    print(f"Mean Fitness: {mean_fitness}")
    print(f"Standard Deviation of Fitness: {std_fitness}")
    print(f"Final Best Fitness: {model.solution[1][0]}")

    # Convergence curve
    import matplotlib.pyplot as plt
    plt.plot(best_fitness_per_iteration)
    plt.xlabel("Iteration")
    plt.ylabel("Best Fitness")
    plt.title("Convergence Curve")
    plt.show()


In [ ]:

# y_pred = rf_model.predict(X_test_std)
# x_print = metrics.r2_score(y_test.values.ravel(), y_predict)
# print(x_print)


In [ ]:

# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"R-squared (R2): {r2:.2f}")